In [1]:
from dolfinx import mesh, fem
from dolfinx.fem.petsc import LinearProblem
import ufl
import numpy as np
import matplotlib.pyplot as plt
from mpi4py import MPI
import utils

In [2]:
width, height = 1.0, 1.0
mesh_density = 32

domain = mesh.create_rectangle(
    MPI.COMM_WORLD,
    [np.array([0, 0]), np.array([width, height])],
    [mesh_density, mesh_density],
)

In [3]:
element = ufl.FiniteElement("P", domain.ufl_cell(), degree=1)
W = fem.FunctionSpace(domain, ufl.MixedElement(element, element))  # P1 FEM space
V, V_to_W = W.sub(0).collapse()

# Trial and test functions
u_i, u_e = ufl.TrialFunctions(W)
v_i, v_e = ufl.TestFunctions(W)

x = ufl.SpatialCoordinate(domain)
I_stim = fem.Function(V)

In [4]:
# Constant
sigma_i, sigma_e = 1.0, 1.0  # intra- and extracellular conductivities
beta = 1.0

Hx = 0.65  # heart locations
Hy = 0.65
R = 0.1


# stimulus
def heart(x):
    return (x[0] < Hx + R) & (x[0] > Hx - R) & (x[1] < Hy + R) & (x[1] > Hy - R)


cells_heart = mesh.locate_entities(domain, 0, heart)
I_stim.x.array[cells_heart] = np.full_like(cells_heart, 1)

In [6]:
# MODEL
a_i = sigma_i * ufl.dot(ufl.grad(u_i), ufl.grad(v_i)) * ufl.dx
a_e = (sigma_i + sigma_e) * ufl.dot(ufl.grad(u_e), ufl.grad(v_e)) * ufl.dx
L_i = beta * I_stim * v_i * ufl.dx
L_e = -beta * I_stim * v_e * ufl.dx
F = a_i + a_e - L_i - L_e

In [7]:
problem = LinearProblem(ufl.lhs(F), ufl.rhs(F), [])
u = problem.solve()
u_i_sol = u.sub(0).collapse()
u_i_sol.x.array[:] = u_i_sol.x.array[:] - np.mean(u_i_sol.x.array)
u_e_sol = u.sub(1).collapse()
u_e_sol.x.array[:] = u_e_sol.x.array[:] - np.mean(u_e_sol.x.array)

utils.plot_function(u_i_sol, V, "u_i")
utils.plot_function(u_e_sol, V, "u_e")

Widget(value='<iframe src="http://localhost:43211/index.html?ui=P_0x7fde29a8a450_0&reconnect=auto" class="pyvi…

Widget(value='<iframe src="http://localhost:43211/index.html?ui=P_0x7fde07fdfe10_1&reconnect=auto" class="pyvi…